In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt
from datetime import datetime

# Define the root directory
# root_dir = r'U:\NVL\HX\A0\results_experimental\ThermalProfile'
root_dir = r'U:\NVL\HX\A1\results_experimental\ThermalProfile'

# Dictionary to store data for each subfolder
subfolder_data = {}

# Function to process each file and store data by subfolder
def process_file(file_path, subfolder_name):
    try:
        # Skip temporary files (starting with ~)
        if os.path.basename(file_path).startswith('~$'):
            return
        # Determine sheet name based on subfolder
        if "GNG" in subfolder_name.upper():  # For GNG-related subfolders
            sheet_name = "ExecuteContent_Marionette_GNG"
        else:
            sheet_name = "SearchVoltage Results"  # Default for HotVmin, etc.

        print(f"Reading {file_path} → Sheet: {sheet_name}")
        
        # Use pd.read_excel directly instead of ExcelFile
        df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
        # required_cols = ['run_order', 'cmvprofiling.Max_DTS_Profile_max', 
        #                'cmvprofiling.Intec_TC_Profile_max', 'cmvprofiling.Intec_FB_Profile_max']
        required_cols = ['run_order', 'cmvprofiling.core_dts_log_max'] #Core
        # required_cols = ['run_order', 'cmvprofiling.cdie_dts_log_max'] #Ring
        # required_cols = ['run_order', 'cmvprofiling.atom_dts_log_max'] #Atom

        if all(col in df.columns for col in required_cols):
            # Filter rows where all required columns have non-null values
            valid_data = df[required_cols].dropna()
            
            # Initialize subfolder data if not present
            if subfolder_name not in subfolder_data:
                # subfolder_data[subfolder_name] = {'run_x': [], 'dts_y': [], 'tc_y': [], 'fb_y': []}
                subfolder_data[subfolder_name] = {'run_x': [], 'dts_y': []}
            
            # Convert to integers with robust handling
            run_x_data = valid_data['run_order'].tolist()
            dts_y_data = pd.to_numeric(valid_data['cmvprofiling.core_dts_log_max'], errors='coerce').dropna().astype(int).tolist() #Core
            # dts_y_data = pd.to_numeric(valid_data['cmvprofiling.cdie_dts_log_max'], errors='coerce').dropna().astype(int).tolist() #Ring
            # dts_y_data = pd.to_numeric(valid_data['cmvprofiling.atom_dts_log_max'], errors='coerce').dropna().astype(int).tolist() #Ring
            
            # tc_y_data = pd.to_numeric(valid_data['cmvprofiling.Intec_TC_Profile_max'], errors='coerce').dropna().astype(int).tolist()
            # fb_y_data = pd.to_numeric(valid_data['cmvprofiling.Intec_FB_Profile_max'], errors='coerce').dropna().astype(int).tolist()
            
            # Debug: Print lengths to check for mismatches
            print(f"File: {file_path}")
            # print(f"Length of run_x: {len(run_x_data)}, dts_y: {len(dts_y_data)}, tc_y: {len(tc_y_data)}, fb_y: {len(fb_y_data)}")
            print(f"Length of run_x: {len(run_x_data)}, dts_y: {len(dts_y_data)}")
            
            # Find the minimum length to trim all lists
            # min_length = min(len(run_x_data), len(dts_y_data), len(tc_y_data), len(fb_y_data))
            min_length = min(len(run_x_data), len(dts_y_data))
            if min_length > 0:
                subfolder_data[subfolder_name]['run_x'].extend(run_x_data[:min_length])
                subfolder_data[subfolder_name]['dts_y'].extend(dts_y_data[:min_length])
                # subfolder_data[subfolder_name]['tc_y'].extend(tc_y_data[:min_length])
                # subfolder_data[subfolder_name]['fb_y'].extend(fb_y_data[:min_length])
            else:
                missing = [c for c in required_cols if c not in df.columns]
                print(f"    Missing columns in {sheet_name}: {missing} – SKIPPED")
    except Exception as e:
        print(f"Error processing {file_path} (sheet: {sheet_name}): {e}")

# Function to calculate temperature range statistics for (high, low) ranges
def calculate_stats(data_series, ranges):
    stats = {f"{high}-{low}": 0 for high, low in ranges}  # Format as high-low
    for temp in data_series:
        for high, low in ranges:
            if low <= temp <= high:  # Adjusted comparison for (high, low) format
                stats[f"{high}-{low}"] += 1
                break
    return stats

# Function to parse timestamp from folder name and get the latest
def get_latest_timestamped_subfolder(base_path):
    timestamped_folders = []
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path) and folder == "HotVmin":
            subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
            for subfolder in subfolders:
                try:
                    # Parse timestamp from folder name (e.g., "2025.07.30_21.48.51")
                    timestamp = datetime.strptime(subfolder, "%Y.%m.%d_%H.%M.%S")
                    timestamped_folders.append((subfolder, timestamp))
                except ValueError:
                    continue
    if timestamped_folders:
        latest_subfolder = max(timestamped_folders, key=lambda x: x[1])[0]
        return os.path.join(base_path, "HotVmin", latest_subfolder)
    return None

# Traverse the directory structure
try:
    main_folders = [f for f in os.listdir(root_dir) if 'U5' in f or 'D4' in f]
    for main in main_folders:
        main_path = os.path.join(root_dir, main)
        # Reset subfolder_data for each main folder to avoid cross-contamination
        subfolder_data.clear()
        
        # Debug: Confirm processing of each main folder
        print(f"Processing main folder: {main}")
        
        # Get immediate subfolders under the main folder
        subfolders = [d for d in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, d))]
        for subfolder in subfolders:
            subfolder_path = os.path.join(main_path, subfolder)
            if subfolder in ["HotVmin", "GNG"]:
                # For HotVmin, process only the latest timestamped subfolder
                latest_hotvmin_path = get_latest_timestamped_subfolder(subfolder_path)
                if latest_hotvmin_path:
                    subfolder_name = os.path.basename(latest_hotvmin_path)
                    for dirpath, dirnames, filenames in os.walk(latest_hotvmin_path):
                        for filename in filenames:
                            if filename.endswith('.xlsx') and ('HotVmin' in filename or 'GNG' in filename):
                                file_path = os.path.join(dirpath, filename)
                                process_file(file_path, subfolder_name)
                else:
                    print(f"No valid timestamped subfolder found in {subfolder_path}")
            else:
                # For other subfolders, process as usual
                for dirpath, dirnames, filenames in os.walk(subfolder_path):
                    subfolder_name = subfolder
                    for filename in filenames:
                        if filename.endswith('.xlsx') and ('HotVmin' in filename or 'GNG' in filename):
                            file_path = os.path.join(dirpath, filename)
                            process_file(file_path, subfolder_name)

        # Create a PowerPoint presentation for the current main folder
        prs = Presentation()
        slide_layout = prs.slide_layouts[6]  # Use blank layout to remove title box

        # Define temperature ranges with largest number first and sort by high value in descending order
        temp_ranges = [(130, 120), (119, 110), (109, 105), (104, 100), (99, 95), (94, 90), (89, 85), (84, 80), (70, 60)]
        temp_ranges.sort(key=lambda x: x[0], reverse=True)  # Sort by first number (high) descending

        # Create a separate plot and slide for each subfolder
        for subfolder, data in subfolder_data.items():
            if any(data.values()):  # Create plot if any list has data
                # Create a figure
                plt.figure(figsize=(18, 8))
                # plt.scatter(data['run_x'], data['dts_y'], color='blue', label='cmvprofiling.Max_DTS_Profile_max')
                plt.scatter(data['run_x'], data['dts_y'], color='blue', label= 'cmvprofiling.core_dts_log_max') #Core
                # plt.scatter(data['run_x'], data['dts_y'], color='blue', label= 'cmvprofiling.cdie_dts_log_max') #Ring
                # plt.scatter(data['run_x'], data['dts_y'], color='blue', label= 'cmvprofiling.atom_dts_log_max') #Atom

                # plt.scatter(data['run_x'], data['tc_y'], color='green', label='cmvprofiling.Intec_TC_Profile_max')
                # plt.scatter(data['run_x'], data['fb_y'], color='red', label='cmvprofiling.Intec_FB_Profile_max')

                plt.title(f'Thermal Profiling - {subfolder}')
                plt.xlabel('run_order')
                plt.ylabel('Temperature')
                plt.legend(loc='upper right', fontsize=8)
                plt.grid(True)
                plt.tight_layout()

                # Save the plot as an image
                image_path = f'temp_plot_{subfolder}.png'
                plt.savefig(image_path, bbox_inches='tight', dpi=300)
                plt.close()  # Close the figure to free memory

                # Add a slide to the PowerPoint
                slide = prs.slides.add_slide(slide_layout)

                # Add the plot image with adjusted position
                pic = slide.shapes.add_picture(image_path, Inches(0.5), Inches(0.5), width=Inches(9))

                # Add statistical table below the plot
                table_left = Inches(0.5)
                table_top = Inches(4.45)  # Adjusted to start lower due to no title
                table_width = Inches(9)
                table_height = Inches(3)  # Height to accommodate 9 ranges
                table = slide.shapes.add_table(rows=10, cols=5, left=table_left, top=table_top, 
                                              width=table_width, height=table_height).table

                # Set table headers
                table.cell(0, 0).text = 'Temperature Range'
                table.cell(0, 1).text = 'Max_DTS'
                # table.cell(0, 2).text = 'TCase'
                # table.cell(0, 3).text = 'FB'
                table.cell(0, 4).text = 'Max_DTS Percentage (%)'
                for cell in table.rows[0].cells:
                    cell.text_frame.paragraphs[0].font.size = Pt(10)
                    cell.text_frame.paragraphs[0].font.bold = True

                # Calculate and populate statistics
                dts_stats = calculate_stats(data['dts_y'], temp_ranges)
                # tc_stats = calculate_stats(data['tc_y'], temp_ranges)
                # fb_stats = calculate_stats(data['fb_y'], temp_ranges)

                # Calculate totals for percentage
                dts_total = sum(dts_stats.values())

                # Populate table with data
                for i, (high, low) in enumerate(temp_ranges, 1):
                    range_str = f"{high}-{low}"
                    table.cell(i, 0).text = range_str
                    table.cell(i, 1).text = str(dts_stats[range_str])
                    # table.cell(i, 2).text = str(tc_stats[range_str])
                    # table.cell(i, 3).text = str(fb_stats[range_str])
                    # Calculate and format percentage for each series
                    dts_pct = (dts_stats[range_str] / dts_total * 100) if dts_total > 0 else 0
                    table.cell(i, 4).text = f"{dts_pct:.1f}%"
                    for cell in table.rows[i].cells:
                        cell.text_frame.paragraphs[0].font.size = Pt(8)

                # Remove the temporary image file
                os.remove(image_path)

        # Define the custom save path for the PowerPoint file with timestamp and main folder name
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  # Format: YYYYMMDD_HHMMSS
        # output_pptx = r'U:\NVL\HX\A0\results_experimental\ThermalProfileSummary\Thermal_Profiling_Results_{}_{}.pptx'.format(main, timestamp)
        output_pptx = r'U:\NVL\HX\A1\results_experimental\ThermalProfileSummary\Thermal_Profiling_Results_{}_{}.pptx'.format(main, timestamp)
        # Ensure the output directory exists
        os.makedirs(os.path.dirname(output_pptx), exist_ok=True)

        # Save the PowerPoint file for the current main folder
        prs.save(output_pptx)
        print(f"PowerPoint file saved as {output_pptx}")

        # Ensure the script ends properly
        print("Script execution completed.")
        
except Exception as e:
    print(f"Error during execution: {e}")

Processing main folder: U538G09400769
Reading U:\NVL\HX\A1\results_experimental\ThermalProfile\U538G09400769\NVL_Hx816_CDIE_C2_EFI_F4_30C_HotGNG\2026.01.07_11.35.01\PGPMK3ECYNL_U538G09400769_NVL_Hx816_CDIE_C2_EFI_F4_HotGNG.xlsx → Sheet: ExecuteContent_Marionette_GNG
Reading U:\NVL\HX\A1\results_experimental\ThermalProfile\U538G09400769\NVL_Hx816_CDIE_C2_EFI_F4_35C_HotGNG\2026.01.06_14.16.10\PGPMK21LJ0J_U538G09400769_NVL_Hx816_CDIE_C2_EFI_F4_HotGNG.xlsx → Sheet: ExecuteContent_Marionette_GNG
Reading U:\NVL\HX\A1\results_experimental\ThermalProfile\U538G09400769\NVL_Hx816_CDIE_C2_EFI_F4_40C_HotGNG\2026.01.06_19.34.42\PGPMK27EQMI_U538G09400769_NVL_Hx816_CDIE_C2_EFI_F4_HotGNG.xlsx → Sheet: ExecuteContent_Marionette_GNG
PowerPoint file saved as U:\NVL\HX\A1\results_experimental\ThermalProfileSummary\Thermal_Profiling_Results_U538G09400769_20260107_115516.pptx
Script execution completed.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pptx import Presentation
from datetime import datetime

# Define the root directory
root_dir = r'U:\NVL\HX\A0\results_experimental\ThermalProfile'

# Dictionary to store data for each subfolder, with data separated by source
subfolder_data = {}

# Function to process each file and store data by subfolder and source
def process_file(file_path, main_folder, subfolder_name):
    try:
        # Skip temporary files (starting with ~) without printing a message
        if os.path.basename(file_path).startswith('~$'):
            return
        
        # Use pd.read_excel directly instead of ExcelFile
        df = pd.read_excel(file_path, sheet_name='SearchVoltage Results', engine='openpyxl')
        required_cols = ['run_order', 'cmvprofiling.Max_DTS_Profile_max', 
                       'cmvprofiling.Intec_TC_Profile_max', 'cmvprofiling.Intec_FB_Profile_max']
        if all(col in df.columns for col in required_cols):
            # Filter rows where all required columns have non-null values
            valid_data = df[required_cols].dropna()
            
            # Initialize subfolder data if not present
            if subfolder_name not in subfolder_data:
                subfolder_data[subfolder_name] = {}
            
            # Initialize source data if not present
            if main_folder not in subfolder_data[subfolder_name]:
                subfolder_data[subfolder_name][main_folder] = {'run_x': [], 'dts_y': [], 'tc_y': [], 'fb_y': []}
            
            # Convert to integers with robust handling
            run_x_data = valid_data['run_order'].tolist()
            dts_y_data = pd.to_numeric(valid_data['cmvprofiling.Max_DTS_Profile_max'], errors='coerce').dropna().astype(int).tolist()
            tc_y_data = pd.to_numeric(valid_data['cmvprofiling.Intec_TC_Profile_max'], errors='coerce').dropna().astype(int).tolist()
            fb_y_data = pd.to_numeric(valid_data['cmvprofiling.Intec_FB_Profile_max'], errors='coerce').dropna().astype(int).tolist()
            
            # Debug: Print lengths to check for mismatches
            print(f"File: {file_path}")
            print(f"Length of run_x: {len(run_x_data)}, dts_y: {len(dts_y_data)}, tc_y: {len(tc_y_data)}, fb_y: {len(fb_y_data)}")
            
            # Find the minimum length to trim all lists
            min_length = min(len(run_x_data), len(dts_y_data), len(tc_y_data), len(fb_y_data))
            if min_length > 0:
                subfolder_data[subfolder_name][main_folder]['run_x'].extend(run_x_data[:min_length])
                subfolder_data[subfolder_name][main_folder]['dts_y'].extend(dts_y_data[:min_length])
                subfolder_data[subfolder_name][main_folder]['tc_y'].extend(tc_y_data[:min_length])
                subfolder_data[subfolder_name][main_folder]['fb_y'].extend(fb_y_data[:min_length])
            else:
                print(f"Warning: No valid data after processing {file_path}.")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Traverse the directory structure
try:
    main_folders = [f for f in os.listdir(root_dir) if 'D3' in f or 'D4' in f]
    for main in main_folders:
        main_path = os.path.join(root_dir, main)
        print(f"Processing main folder: {main}")
        
        # Get immediate subfolders under the main folder
        subfolders = [d for d in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, d))]
        for subfolder in subfolders:
            subfolder_path = os.path.join(main_path, subfolder)
            for dirpath, dirnames, filenames in os.walk(subfolder_path):
                subfolder_name = subfolder
                for filename in filenames:
                    if filename.endswith('.xlsx') and ('HotVmin' in filename or 'HotGNG' in filename):
                        file_path = os.path.join(dirpath, filename)
                        process_file(file_path, main, subfolder_name)

    # Create a single PowerPoint presentation for all data
    prs = Presentation()
    slide_layout = prs.slide_layouts[5]  # Use a layout with a title and content

    # Define color sets for D3 and D4 main folders
    color_sets = {
        'D3': {
            'dts_y': 'darkblue',
            'tc_y': 'darkgreen',
            'fb_y': 'darkred'
        },
        'D4': {
            'dts_y': 'blue',
            'tc_y': 'green',
            'fb_y': 'red'
        }
    }

    # Create a separate plot and slide for each subfolder, plotting data by source
    for subfolder, sources_data in subfolder_data.items():
        # Check if any source has data
        has_data = any(len(data['run_x']) > 0 for data in sources_data.values())
        if has_data:  # Create plot if any source has data
            plt.figure(figsize=(18, 8))
            for source, data in sources_data.items():
                if data['run_x']:  # Only plot if this source has data
                    colors = color_sets.get(source[:2], color_sets['D3'])  # Get color set based on source prefix
                    plt.scatter(data['run_x'], data['dts_y'], color=colors['dts_y'], label=f'{source} (DTS)', alpha=0.6)
                    plt.scatter(data['run_x'], data['tc_y'], color=colors['tc_y'], label=f'{source} (TC)', alpha=0.6)
                    plt.scatter(data['run_x'], data['fb_y'], color=colors['fb_y'], label=f'{source} (FB)', alpha=0.6)

            plt.title(f'Thermal Profiling - {subfolder}')
            plt.xlabel('run_order')
            plt.ylabel('Temperature')
            plt.legend(loc='upper right', fontsize=8, bbox_to_anchor=(1.15, 1))
            plt.grid(True)
            plt.tight_layout()

            # Save the plot as an image
            image_path = f'temp_plot_{subfolder.replace("/", "_")}.png'  # Replace / with _ to avoid file path issues
            plt.savefig(image_path, bbox_inches='tight', dpi=300)
            plt.close()  # Close the figure to free memory

            # Add a slide to the PowerPoint
            slide = prs.slides.add_slide(slide_layout)
            title = slide.shapes.title
            title.text = f'Thermal Profiling - {subfolder}'
            pic = slide.shapes.add_picture(image_path, 50, 100, width=prs.slide_width - 100)  # Adjust position and size

            # Remove the temporary image file
            os.remove(image_path)

    # Define the custom save path for the single PowerPoint file with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  # Format: YYYYMMDD_HHMMSS
    output_pptx = r'U:\NVL\HX\A0\results_experimental\ThermalProfileSummary\Thermal_Profiling_Merge_Results_{}.pptx'.format(timestamp)
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_pptx), exist_ok=True)

    # Save the single PowerPoint file
    prs.save(output_pptx)
    print(f"PowerPoint file saved as {output_pptx}")

    # Ensure the script ends properly
    print("Script execution completed.")

except Exception as e:
    print(f"Error during execution: {e}")